In [1]:
from utils_warehouse import get_all_fbs_acts, create_insert_table_db
import pandas as pd

In [2]:
# Получаем все акты-приема передачи и ФБО и ФБС
df = get_all_fbs_acts()

# Из полученных данных формируем акты-приема передачи для ФБО
fbo_acts_df = df[['№ п\п', 'Товар (наименование)',  'Ед. изм.', 'Фактически принято - баркод', ' - артикул продавца', ' - сорт, размер', ' - КИЗ', ' - ШК короба', ' - кол-во', 'Документ','Номер_документа', 'Дата', 'account']]
# ВБ иногда путает акты ФБО и ФБС, поэтому фильтруем по ШК короба
fbo_acts_df = fbo_acts_df[fbo_acts_df[' - ШК короба'].notna()]
# Приводим названия колонок к читаемому виду
fbo_acts_df = fbo_acts_df.rename(columns={
    '№ п\\п': 'num',
    'Товар (наименование)': 'product_name',
    'Ед. изм.': 'unit',
    'Фактически принято - баркод': 'barcode',
    ' - артикул продавца': 'vendor_code',
    ' - сорт, размер': 'size',
    ' - КИЗ': 'kiz',
    ' - ШК короба': 'box_barcode',
    ' - кол-во': 'quantity',
    'Документ': 'document',
    'Номер_документа': 'document_number',
    'Дата': 'date',
    'account': 'account'
})
# Заменяем пустоты
fbo_acts_df['kiz'] = fbo_acts_df['kiz'].fillna('Нет КИЗов')
# Приводим колонку с датой к нужному формату
fbo_acts_df['date'] = fbo_acts_df['date'].str.replace('"','').str.replace(' ', '').str.replace('г.', '')
fbo_acts_df['date'] = pd.to_datetime(fbo_acts_df['date'], format='%d%m%Y')
# Удаляем лишние символы из номера документа
fbo_acts_df['document_number'] = fbo_acts_df['document'].str.extract(r'(\d+)\.zip')[0]
print('Данные по ФБО полчены')

# Из полученных данных формируем акты-приема передачи для ФБС
fbs_acts_df = df[['№ п\п', 'Номер заказа', 'Ед. изм.', 'Фактически принято - Стикер/этикетка', ' - Кол-во', 'Документ','Номер_документа', 'Дата', 'account']]
# ВБ иногда путает акты ФБО и ФБС, поэтому фильтруем по ШК короба
fbs_acts_df = fbs_acts_df[fbs_acts_df['Фактически принято - Стикер/этикетка'].notna()]
# Приводим названия колонок к читаемому виду
fbs_acts_df = fbs_acts_df.rename(columns={
    '№ п\\п': 'num',
    'Номер заказа': 'order_number',
    'Ед. изм.': 'unit',
    'Фактически принято - Стикер/этикетка': 'sticker',
    ' - Кол-во': 'quantity',
    'Документ': 'document',
    'Номер_документа': 'document_number',
    'Дата': 'date',
    'account': 'account'
})

print('Данные по ФБС полчены')

Файлы внутри вложенного ZIP:
  - act-income-mp-172966691.xlsx
Количество колонок в df: 5
Количество заголовков: 5
Данные по act-income-mp-172966691.zip добавлены в список
  - signature.sig
Файлы внутри вложенного ZIP:
  - act-income-mp-173004631.xlsx
Количество колонок в df: 5
Количество заголовков: 5
Данные по act-income-mp-173004631.zip добавлены в список
  - signature.sig
Файлы внутри вложенного ZIP:
  - act-income-mp-173142128.xlsx
Количество колонок в df: 5
Количество заголовков: 5
Данные по act-income-mp-173142128.zip добавлены в список
  - signature.sig
Файлы внутри вложенного ZIP:
  - act-income-mp-173020681.xlsx
Количество колонок в df: 5
Количество заголовков: 5
Данные по act-income-mp-173020681.zip добавлены в список
  - signature.sig
Файлы внутри вложенного ZIP:
  - act-income-mp-173137618.xlsx
Количество колонок в df: 5
Количество заголовков: 5
Данные по act-income-mp-173137618.zip добавлены в список
  - signature.sig
Файлы внутри вложенного ZIP:
  - act-income-mp-17312251

In [3]:
columns_type_fbo = {
    'num': 'INTEGER',
    'product_name': 'VARCHAR(255)',
    'unit': 'VARCHAR(50)',
    'barcode': 'VARCHAR(50)',
    'vendor_code': 'VARCHAR(100)',
    'size': 'VARCHAR(50)',
    'kiz': 'VARCHAR(255)',
    'box_barcode': 'VARCHAR(50)',
    'quantity': 'INTEGER',
    'document': 'VARCHAR(255)',
    'document_number': 'VARCHAR(50)',
    'date': 'DATE',
    'account': 'VARCHAR(100)'
}

columns_type_fbs = {
    'num': 'INTEGER',
    'order_number': 'VARCHAR(50)',
    'unit': 'VARCHAR(50)',
    'sticker': 'VARCHAR(100)',
    'quantity': 'INTEGER',
    'document': 'VARCHAR(255)',
    'document_number': 'VARCHAR(50)',
    'date': 'DATE',
    'account': 'VARCHAR(100)'
}

table_name_fbo = 'acceptance_fbo_acts'
key_columns_fbo = ('vendor_code', 'box_barcode', 'document_number')
table_name_fbs = 'acceptance_fbs_acts'
key_columns_fbs = ('order_number', 'sticker', 'document_number')


create_insert_table_db(fbo_acts_df, table_name_fbo, columns_type_fbo, ('vendor_code', 'box_barcode', 'document_number'))
create_insert_table_db(fbs_acts_df, table_name_fbs, columns_type_fbs, key_columns_fbs)
print("Данные по ФБО и ФБС успешно загружены в базу данных.")


Соединение с базой данных установлено.
Таблица acceptance_fbo_acts создана или уже существует.
Данные успешно добавлены в БД.
Соединение с базой данных закрыто.
Соединение с базой данных установлено.
Таблица acceptance_fbs_acts создана или уже существует.
Ошибка при загрузке данных в БД: date/time field value out of range: "31.07.2025"
LINE 3: ...589,'1','act-income-mp-171229017.zip','171229017','31.07.202...
                                                             ^
HINT:  Perhaps you need a different "datestyle" setting.

Соединение с базой данных закрыто.
Данные по ФБО и ФБС успешно загружены в базу данных.
